# Algorithmic_Trading_Machine_Learning_Quant_Strategies

All Packages Needed:
- pandas, numpy, matplotlib, statsmodels, pandas_datareader, datetime, yfinance, sklearn, PyPortfolioOpt

In [2]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt 
import statsmodels.api as sm 
import pandas as pd 
import numpy as np 
import datetime as dt 
import yfinance as yf 
#import pandas_ta 
import warnings
warnings.filterwarnings('ignore')

# 1. Download/Load SP500 stock prices data.

In [15]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')  # some tickers with '.' could cause issues later
 
symbols_list = sp500['Symbol'].unique().tolist() # not suvivorship-bias free

end_date = '2024-01-03'
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date)
df

[*********************100%%**********************]  503 of 503 completed


Adj Close                         ...   Volume                  
                     A        AAL        AAPL  ...     ZBRA     ZION      ZTS
Date                                           ...                           
2016-01-05   38.003239  38.722759   23.376621  ...   611300  2653600  3116700
2016-01-06   38.171921  39.401264   22.919144  ...   881900  4164200  4670500
2016-01-07   36.550583  38.655853   21.951853  ...  1210600  4234000  4324400
2016-01-08   36.166336  38.579414   22.067926  ...  1034700  3401600  4946100
2016-01-11   35.557144  39.257923   22.425253  ...  1034700  4627200  4306900
...                ...        ...         ...  ...      ...      ...      ...
2023-12-26  139.573929  14.110000  193.050003  ...   220000  1131600   814600
2023-12-27  139.583923  13.990000  193.149994  ...   275700  1345100   766400
2023-12-28  139.534012  13.980000  193.580002  ...   193900  1125900   880100
2023-12-29  139.029999  13.740000  192.529999  ...   251300  1420600  1007200
2024-01-02  138.750000  13.440000  185.639999  ...   452900  2256700  1642300

[2012 rows x 3018 columns]

the above object has a multi-index column: e.g. Adj Close : [A,AAL,AAPL,...], Volume: [...]
also there's a ton of columns & this is inefficient. to fix this:

In [16]:
df = df.stack()
df

Adj Close       Close  ...        Open       Volume
Date                                     ...                         
2016-01-05 A      38.003239   40.549999  ...   40.730000    2587200.0
           AAL    38.722759   40.520000  ...   41.180000   10514200.0
           AAPL   23.376621   25.677500  ...   26.437500  223164000.0
           ABBV   40.608185   57.369999  ...   57.990002    7179600.0
           ABT    36.947697   42.919998  ...   42.959999    8245200.0
...                     ...         ...  ...         ...          ...
2024-01-02 YUM   129.029999  129.029999  ...  129.729996    1485300.0
           ZBH   121.389999  121.389999  ...  121.019997    1087400.0
           ZBRA  267.980011  267.980011  ...  268.559998     452900.0
           ZION   44.139999   44.139999  ...   43.500000    2256700.0
           ZTS   196.570007  196.570007  ...  195.789993    1642300.0

[994528 rows x 6 columns]

In [17]:
df.index.names = ['date','ticker']
df

Adj Close       Close  ...        Open       Volume
date       ticker                          ...                         
2016-01-05 A        38.003239   40.549999  ...   40.730000    2587200.0
           AAL      38.722759   40.520000  ...   41.180000   10514200.0
           AAPL     23.376621   25.677500  ...   26.437500  223164000.0
           ABBV     40.608185   57.369999  ...   57.990002    7179600.0
           ABT      36.947697   42.919998  ...   42.959999    8245200.0
...                       ...         ...  ...         ...          ...
2024-01-02 YUM     129.029999  129.029999  ...  129.729996    1485300.0
           ZBH     121.389999  121.389999  ...  121.019997    1087400.0
           ZBRA    267.980011  267.980011  ...  268.559998     452900.0
           ZION     44.139999   44.139999  ...   43.500000    2256700.0
           ZTS     196.570007  196.570007  ...  195.789993    1642300.0

[994528 rows x 6 columns]

In [19]:
df.columns = df.columns.str.lower()
df

adj close       close  ...        open       volume
date       ticker                          ...                         
2016-01-05 A        38.003239   40.549999  ...   40.730000    2587200.0
           AAL      38.722759   40.520000  ...   41.180000   10514200.0
           AAPL     23.376621   25.677500  ...   26.437500  223164000.0
           ABBV     40.608185   57.369999  ...   57.990002    7179600.0
           ABT      36.947697   42.919998  ...   42.959999    8245200.0
...                       ...         ...  ...         ...          ...
2024-01-02 YUM     129.029999  129.029999  ...  129.729996    1485300.0
           ZBH     121.389999  121.389999  ...  121.019997    1087400.0
           ZBRA    267.980011  267.980011  ...  268.559998     452900.0
           ZION     44.139999   44.139999  ...   43.500000    2256700.0
           ZTS     196.570007  196.570007  ...  195.789993    1642300.0

[994528 rows x 6 columns]

# 2. Calculate features and technical indicators for each stock.

- Garman-Klass Volatility
- RSI
- Bollinger Bands
- ATR
- MACD
- Dollar Volume

Garman-Klass Volatility = ((ln(High) - ln(Low))^2)/2 - (2ln(2) - 1)(ln(Adj Close) - ln(Open))^2